#### Packages installation 

In [1]:
!pip install "git+https://github.com/panalexeu/horchunk.git"

  Cloning https://github.com/panalexeu/horchunk.git to /tmp/pip-req-build-e4a29xkq
  Running command git clone --filter=blob:none --quiet https://github.com/panalexeu/horchunk.git /tmp/pip-req-build-e4a29xkq
  Resolved https://github.com/panalexeu/horchunk.git to commit fc1b83f360cbaa09b06509276a77acc6711346b6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


#### Tuning

In [2]:
from horchunk.chunkers import WindowTuner 

/home/oleksii/python/semantic_chunking/semchunk/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
